# Soccer Mobile
<!-- Introduction of the project -->

## Introduction

In this project the goal is to combine vision, path planning, local navigation, and filtering in order to make Thymio robot navigate trough a map towards a goal.  

As the constraints are established we were free to chose our own implementation. To begin with, a camera is used in order to provide the vision information. The vision module works along with the global path and filter modules extracting estimating and computing the necessary map information, including the robot pose, robot pose, map, static obstacles, and the goal position. 

Indeed, a Kalman filter performs the estimations of the robot pose. Afterwards, the A* algorithm computes the optimal path. Following this, a global controller gives instructions to the motors for them to follow the optimal path. Finally, a local navigation module is implemented in order to provide a vision-free information to Thymio for the most spontaneous events such as a dynamic obstacle.

**Imports**

In [ ]:
import tdmclient.notebook
await tdmclient.notebook.start()
from Thymio import Thymio
import time
import numpy as np

from geo import *
#from Thymio import Thymio
import vision
#import filtering
import global_navigation

## Vision


In [ ]:
# ...
gmap = None
pBall = Pos() #
pGoal = Pos() #

## Global Navigation
Plan a path from the start to the goal.
Return: Waypoints

In [ ]:
planner =  PathPlanner(gmap)

## Local Navigation


The local navigation module allows to take advantage of the proximity sensors located on the five front horizontal proximity sensors. The objective is to bypass the unknown local obstacle for further re-computing of the controller to correct Thymio's speed.

- **Input**

    - Horizontal proximity sensor values


- **Output**

    - Motion control command for robot translation and rotation

**Parameters**

| Name                | Meaning                                                      | Type (Unit) | Global |
| :------------------- | :------------------------------------------------------------ | :----------- | :------ |
| `max_speed`        |  Nominal speed                 | Int         |  |
| `obstThrL`      | Low obstacle threshold to switch state 1->0                | Int         |    |
| `obstThrH` | High obstacle threshold to switch state 0->1                    | Int         |   |
| `obstSpeedGain`         | Variation of speed according to the distance of obstacle | Int |   |
| `state`         | 0=global navigation, 1=local navigation | Bool |   |
| `obst`         | Measurements from left and right prox sensors | Int |   |


**Functions**

Two functions are basically doing the local avoidance. obs is meant to detect an obstacle and return the new state of the robot as stated in the parameters tabular. obstacle_avoidance updates the new speed to return to thymio. It will return the inputs to the motors function. 

In [1]:
@tdmclient.notebook.sync_var
def obs():
    global prox_horizontal, state, obst, obstThrH, obstThrL
    obst = [prox_horizontal[0], prox_horizontal[4]]
    
    if state == 0: 
        # switch from goal tracking to obst avoidance if obstacle detected
        if (obst[0] > obstThrH):
            state = 1
        elif (obst[1] > obstThrH):
            state = 1
    elif state == 1:
        if obst[0] < obstThrL:
            if obst[1] < obstThrL : 
                # switch from obst avoidance to goal tracking if obstacle got unseen
                state = 0
    return state
    
@tdmclient.notebook.sync_var
def obstacle_avoidance():
    global prox_horizontal, state, obst, obstThrH, obstThrL, obstSpeedGain, speed0, speedGain 
    obst = [prox_horizontal[0], prox_horizontal[4]]
    
    # obstacle avoidance: accelerate wheel near obstacle
    left_speed = max_speed + obstSpeedGain * (obst[0] // 100)
    right_speed = max_speed + obstSpeedGain * (obst[1] // 100)
    motors(left_speed, right_speed)

## Filtering
tell where Thymio is.

## Motion Control

The motion control of Thymio is where all movement functions and interations function with thymio are written. They alllow to update the thymio's movment based on the information returned by the global planing.

In [ ]:
controller = MotionController()
control_cycle = 10 # ms = 100Hz

In [ ]:
get_ball = False
# 1. get the ball
planner.set_goal = pBall
planner.set_start = None # Pos of Thymio [Filtering]
waypoints = planner.plan()
while not get_ball:
    if controller.obs_forward():
        pass #   [Local Navigation]
        #          return to waypoints to follow
    else:
        waypoints = controller.go_pos(waypoints) # follow the waypoints 
    pass

# 2. Push the ball to the goal
finished = False
planner.set_goal = pGoal
planner.set_start = None # Pos of Thymio [Filtering]
waypoints = planner.plan()
while not finished:
    if controller.obs_forward():
        pass #   [Local Navigation]
        #          return to waypoints to follow
    else:
        waypoints = controller.go_pos(waypoints) # follow the waypoints 
    pass

## Main : the soccer player

In this main all the motion control and local avoidance functions are called in a while loop. This merges all the modules and communicates with the thymio thanks to the motion contol module.

In [ ]:
G_camera_timer = time.time()
G_track_timer = time.time()

def localizate():
    """Track Where Thymio is"""
    global G_camera_timer
    starter = G_filter.timer
    # 3. Localization 
    # 3.1 odometer
    dsl, dsr = get_displacement()
    # 3.2 With Vision
    if starter - G_camera_timer > S_camera_interval:
        vision_thymio_state = G_vision.getThymio()
        G_camera_timer = starter
        G_filter.kalman_filter(dsr, dsl, vision_thymio_state)
    else:        
        G_filter.kalman_filter(dsr, dsl)
    
    thymio_state = G_filter.get_state()
    return thymio_state

In [ ]:
def main():
    """Main Program"""
    # 1. initialize everythings
    # 1.1 The Map
    #G_map = G_vision.getMap()
    #Ball_pos = G_vision.getBall()
    #Gate_pos = G_vision.getGate()
    #vision_thymio_state = G_vision.getThymio()
    #G_pp = global_navigation.PathPlanner(G_map, method="A*", neighbor=8, simplify = True)
    # 1.2 Where I am
    #for _ in range(S_stablize_filter_steps):
        #Thymio_state = G_filter.kalman_filter(0, 0, vision_thymio_state)

    # 2. main loop of 2 tasks
    #for goal in ["ball", "gate"]:
    #    # 2.1 Task set up
    #    if goal == "ball":
    #       Goal_state = G_pp.approach(Ball_pos)
    #    else:
    #        Goal_state = G_pp.approach(Gate_pos)
    #    G_pp.set_goal(Goal_state.pos)
    #    G_pp.set_start(Thymio_state)
    #    path = G_pp.plan()
    #    Goal_state = Goal_state.multiply(G_map.scale)
    #    Global_path = G_pp.assign_ori(path, Goal_state.ori)
        # 2.2 Tackle the task
    #while True:
    #    starter = time.time()
    #    # 3. Localization
    #    Thymio_state = localizate()
    #    # 2.2.1 Finished?
    #    if Thymio_state.dis(Goal_state) < S_epsilon_dis \
    #        and abs(Thymio_state.ori - Goal_state.ori) < S_epsilon_theta:
    #        if G_verbose:
    #            print("Terminate Reached!")
    #        break
    #    # 2.2.2 Is there obstacles on the front?

    state = obs()
    time.sleep(0.2)

    if state == 1:
        print(state)
        obstacle_avoidance()

    else:
        print(state)
        motors(max_speed, max_speed)
    #        if starter - G_track_timer > S_track_interval:
    #            # 4. Follow the path    # <-- The only task can run under low frequency
    #            reached = path_tracking(Global_path[0], Thymio_state, len(Global_path) == 1)
    #            if reached:
    #                Global_path = Global_path[1:]
    #                # assume Global_path is not empty because of 2.2.1
    #            G_track_timer = starter
    #    loop_time = time.time() - starter
    #    # TODO: how long does it takes? -- about 1ms
    #    #       then, which motor speed fit the period time
    #    #             should we stop to calibrate pose?
    #    if G_verbose:
    #        print(F"looper time: {loop_time*1000 :.0f}ms")
    
    while True:
        main()

In [ ]:
motors(0,0)